In [ ]:
#Importo le librerie necessarie
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
from glob import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pickle

In [ ]:
#Definisco il percorso del dataset
percorso_dataset = "/content/drive/MyDrive/prova immagini/Raccolta_differenziata3"
#Leggo le sotto cartelle
categorie = os.listdir(percorso_dataset)

#Definisco una lista data
data = []
#Per ogni cartella presente nell'insieme delle cartelle
for categoria in categorie:
    #Definisco il singolo percorso
    percorso_categoria = os.path.join(percorso_dataset, categoria)
    #Leggo tutte le immagini presenti
    immagini = glob(os.path.join(percorso_categoria, "*.jpg"))

    #Per ogni percorso di immagine
    for percorso_immagine in immagini:
      #Salvo all'interno della lista data il percorso dell'immagine e la relativa categoria
        data.append((percorso_immagine, categoria))

#Definisco un dataframe che definisca la categoria per ogni immagine
df = pd.DataFrame(data, columns=["percorso_immagine", "etichetta"])
#Stampo le prime righe del dataframe
print(df.head())

In [ ]:
#Stabilisco l'indice delle classi
classi_e_indici = {classe: i for i, classe in enumerate(df["etichetta"].unique())}


print("Classi -> Indici:", classi_e_indici)


In [ ]:
#Carico e preprocesso l'immagine
def carico_e_preprocesso_immagine(percorso_immagine, misura_target=(224, 224)):
    #Carico l'immagine definendone percorso e misura
    immagine = load_img(percorso_immagine, target_size=misura_target)
    #Normalizzo l'immagine
    immagine_normalizzata = img_to_array(immagine) / 255.0
    #Restituisco l'immagine normalizzata
    return immagine_normalizzata

#Carico l'immagine normalizzata nel dataframe
df["immagine_normalizzata"] = df["percorso_immagine"].apply(carico_e_preprocesso_immagine)
#Stampo le prime righe del dataset
print(df.head())

In [ ]:
#Mostro le prime immagini del dataset
plt.figure(figsize=(10,5))
#Determino la griglia in cui saranno visualizzate
for i in range(6):
  #Carico le immagini
    immagine = load_img(df["percorso_immagine"][i])
    #Determino una griglia 3x3
    plt.subplot(2, 3, i+1)
    #Mostro l'immagine
    plt.imshow(immagine)
    #Inserisco il titolo di ogni immagine
    plt.title(df["etichetta"][i])
    #Nascondo gli assi
    plt.axis("off")
#Mostro le immagini
plt.show()

In [ ]:
#Divido il dataset in set di train e di test e validazione (30%)
train_df, test_e_validazione_df = train_test_split(df, test_size=0.3, stratify=df["etichetta"], random_state=42)

#Divido il daset di test e validazione a metà definendo un validation set e un test set
validazione_df, test_df = train_test_split(test_e_validazione_df, test_size=0.5, stratify=test_e_validazione_df["etichetta"], random_state=42)

#Stampo le dimensioni dei vari set
print(f"Training Set: {len(train_df)} immagini")
print(f"Validation Set: {len(validazione_df)} immagini")
print(f"Test Set: {len(test_df)} immagini")

In [ ]:
#Preprocesso ogni immagine del dataframe e la converte in array numpy
X_train = np.array([preprocess_image(img) for img in train_df["percorso_immagine"]])
X_val = np.array([preprocess_image(img) for img in validazione_df["percorso_immagine"]])
X_test = np.array([preprocess_image(img) for img in test_df["percorso_immagine"]])

In [ ]:
#Converto le etichette in numeri interi
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df["etichetta"])
y_validazione = label_encoder.transform(validazione_df["etichetta"])
y_test = label_encoder.transform(test_df["etichetta"])

In [ ]:
#Converto le etichette in One Hot Encoding
y_train = tf.keras.utils.to_categorical(y_train)
y_validazione = tf.keras.utils.to_categorical(y_validazione)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
#Determino il modello CNN
modello_cnn = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='softmax')
])

In [ ]:
#Compilo il modello
modello_cnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Mostro il modello
modello_cnn.summary()

In [ ]:
#Addestro il modello
history = modello_cnn.fit(
    X_train, y_train,
    epochs=30,
    batch_size=16,
    validation_data=(X_val, y_validazione)
)

In [ ]:
test_loss, test_acc = modello_cnn.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

In [ ]:
#Percorso immagine
img_path = "/content/drive/MyDrive/prova immagini/prova_biologico.jpg"

#Preprocessamento immagine
def preprocess_image(img_path):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

img_processed = preprocess_image(img_path)

#Mostra immagine
plt.imshow(load_img(img_path, target_size=(224, 224)))
plt.axis("off")
plt.show()

In [ ]:
#Predizione sulla base dell'immagine
pred = modello_cnn.predict(img_processed)

In [ ]:
etichette_classi = sorted(os.listdir(percorso_dataset))

In [ ]:
indice_classe_probabilità_più_alta = np.argmax(pred)

In [ ]:
classe_predetta = etichette_classi[indice_classe_probabilità_più_alta]
print("Classe predetta:", classe_predetta)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
modello_cnn.save('/content/drive/MyDrive/prova immagini/Modello Classificazione Rifiuti.h5')